### Import packages

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
import numpy as np
import os
from tqdm import tqdm
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms
from datasets import load_dataset,DatasetDict,load_from_disk
from torch.utils.data import DataLoader
import torch.nn.functional as F
from itertools import islice
import copy
from torchmetrics.retrieval import RetrievalMAP, RetrievalPrecision
import os
import random
import torch
import torch.nn.functional as F
import torch.nn as nn



### Set seed

In [2]:
main_path = '/Users/aakashagarwal/Downloads/ir_assignment2/'

In [3]:
torch.manual_seed(0)

### Set device

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
    
device

'mps'

In [5]:
mydataset = "evanarlian/imagenet_1k_resized_256"

#### Load dataset

In [6]:
ds = load_dataset(mydataset)
ds = ds.rename_column("image", "img")

ds = DatasetDict({
    'train': ds['train'],
    'test': ds['val']
})

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [7]:
main_path = '/Users/aakashagarwal/Downloads/ir_assignment2/'

#### Test on small dataset

In [8]:
# ds_train = ds['train'].select(range(1000))
# ds_test = ds['test'].select(range(100))

# ds = DatasetDict({
#     'train': ds_train,
#     'test': ds_test
# })

# ds

### Model load

In [9]:
processor = AutoImageProcessor.from_pretrained("google/vit-large-patch32-384",use_fast=True)
model = AutoModelForImageClassification.from_pretrained("google/vit-large-patch32-384")

### Extract features of training data

#### Feature function

In [10]:
# Set model to evaluation mode
model.eval()

transform = transforms.Compose([
        transforms.Resize((384, 384)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert PIL images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Define a function to process and extract features from each image
def extract_features(batch):
    imgs = batch['img']  # This will now be a list of images in a batch

    # Preprocess each image in the batch
    img_tensors = [transform(img).unsqueeze(0).to(device) for img in imgs]  # Add batch dimension and move to device
    img_tensors = torch.cat(img_tensors, dim=0)  # Concatenate to create a batch tensor

    inputs = {'pixel_values': img_tensors}  # Adjust for model input (assuming using Hugging Face models)

    # Forward pass through the model to get features
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    hidden_states = outputs.hidden_states  # List of all hidden states

    # Extract the final hidden state
    final_hidden_state = hidden_states[-1]  # Shape: [batch_size, num_patches, hidden_size]
    
    # Optionally, pool the features
    pooled_features = final_hidden_state.mean(dim=1)  # Shape: [batch_size, hidden_size]
    
    # Return the features as a new column for the batch
    return {"features": pooled_features.cpu().numpy()}  # 


#### Create features of train data or load from saved file.

In [11]:

output_dir = main_path+ f"features/imagenet1k/"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "vit_large/")  # Saving in Arrow format

# Check if features already exist
if os.path.exists(output_file):
    print(f"Features file already exists at {output_file}. Loading features...")
    # Load existing features
    ds_feature = load_from_disk(output_file)
else:
    print("Features file does not exist. Extracting features...")

    # Use `map` to apply the feature extraction across the entire dataset
    ds_feature = ds.map(extract_features, batched=True, batch_size=64)

    # Save the dataset with the new features to disk
    ds_feature.save_to_disk(output_file)  # Saving in the Arrow format
    print("Features saved successfully!")

Features file already exists at /Users/aakashagarwal/Downloads/ir_assignment2/features/imagenet1k/vit_large/. Loading features...


Loading dataset from disk:   0%|          | 0/22 [00:00<?, ?it/s]

In [12]:
ds_feature
ds_feature = ds_feature.rename_column("labels", "label")

#### Collate function

In [13]:
def custom_collate(batch):
    # Define the transformations you want to apply to the images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert PIL images to tensors
    ])

    # Extracting each field from the batch
    index = [item['index'] for item in batch]
    # images = [transform(item['img']) for item in batch]  # Load and transform images
    labels = torch.tensor([item['label'] for item in batch])          # Convert labels to a tensor
    features = [torch.tensor(item['features']) for item in batch]     # Convert features to tensors
    
    # Return a dictionary with batched data
    return {
        'index': index,
        # 'img': torch.stack(images),    # Stack image tensors into a single tensor
        'label': labels,
        'features': torch.stack(features),  # Stack feature tensors into a single tensor
    }


In [14]:
def random_collate(batch):
    # Define the transformations you want to apply to the images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert PIL images to tensors
    ])

    # Extracting each field from the batch
    index = [item['index'] for item in batch]
    images = [transform(item['img']) for item in batch]  # Load and transform images
    labels = torch.tensor([item['label'] for item in batch])          # Convert labels to a tensor
    features = [torch.tensor(item['features']) for item in batch]     # Convert features to tensors
    
    # Return a dictionary with batched data
    return {
        'index': index,
        'img': torch.stack(images),    # Stack image tensors into a single tensor
        'label': labels,
        'features': torch.stack(features),  # Stack feature tensors into a single tensor
    }



#### Dataloading


In [50]:
train_dataloader = DataLoader(ds_feature['train'], batch_size=64, shuffle=True,collate_fn=custom_collate)
test_dataloader = DataLoader(ds_feature['test'], batch_size=64, shuffle=True,collate_fn=custom_collate)

In [37]:
for i,batch in enumerate(test_dataloader):
    if i ==19:
        print(batch['index'])
        break

[4334, 2579, 4290, 2130, 4901, 2570, 4533, 2299, 2250, 3946, 4444, 2047, 2395, 2980, 104, 2040, 3374, 2973, 352, 1682, 3621, 3129, 2499, 3020, 2826, 3974, 1247, 4998, 2341, 3292, 423, 3449, 4273, 1609, 605, 992, 123, 1277, 2498, 4037, 3930, 3615, 3156, 1779, 4608, 2812, 1667, 2100, 3123, 3207, 2208, 995, 2928, 4994, 2700, 885, 4677, 4699, 1872, 3995, 3252, 4565, 3148, 3858]


#### Dataloader to random a random batch, shuffle=True

In [38]:
random_train_dataloader = DataLoader(
    ds_feature['train'], batch_size=64, shuffle=True, collate_fn=random_collate, pin_memory=True
)

### Get radom batch to sample negatives

In [39]:
def get_random_batch():
    random_train_dataloader = DataLoader(
        ds_feature['train'], batch_size=64, shuffle=True, collate_fn=custom_collate, pin_memory=True
    )
    # Get the first batch from the DataLoader
    first_batch = next(iter(random_train_dataloader))
    return first_batch

### HashNetwok Model

In [40]:
import torch
import torch.nn as nn

class HashNetwork(nn.Module):
    def __init__(self, input_dim, hash_bits):
        super(HashNetwork, self).__init__()
        
        # Define the hashing network layers
        self.hashing_network = nn.Sequential(
            nn.Linear(input_dim, input_dim, bias=True),
            nn.ReLU(),
            nn.Dropout(0.3),  # Increased dropout rate for regularization
            nn.Linear(input_dim, input_dim // 2, bias=True),
            nn.ReLU(),
            nn.Linear(input_dim // 2, input_dim // 4, bias=True),
            nn.ReLU(),
            nn.Linear(input_dim // 4, hash_bits, bias=True),
            nn.Tanh()
        )
        
        # Initialize the loss coefficients as a learnable parameter
        self.loss_coefficients = nn.Parameter(torch.FloatTensor(3))
        nn.init.uniform_(self.loss_coefficients, 0.01, 0.1)  # Use uniform initialization for loss coefficients

        # Initialize network weights using Xavier initialization
        self._initialize_weights()

    def _initialize_weights(self):
        for module in self.hashing_network:
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # Apply Xavier initialization to weights
                if module.bias is not None:
                    nn.init.zeros_(module.bias)  # Initialize biases to zero

        # Normalize the loss coefficients to ensure they sum to 1 initially
        self.loss_coefficients.data /= self.loss_coefficients.data.sum()

    def forward(self, x):
        return self.hashing_network(x)

    def get_loss_coefficients(self):
        # Apply softmax to ensure they sum to 1
        return nn.functional.softmax(self.loss_coefficients, dim=0)

    def clamp_and_normalize_loss_coefficients(self, min_value=0.01):
        # Clamp the coefficients to be at least `min_value`
        self.loss_coefficients.data = torch.clamp(self.loss_coefficients.data, min=min_value)
        
        # Normalize the coefficients to sum to 1
        self.loss_coefficients.data /= torch.sum(self.loss_coefficients.data)


### Loss

In [41]:
def custom_loss_function(model, batch_features, batch_labels, random_batch_features, random_batch_labels, 
                         use_hyperparameter=True, alpha=0.01, beta=0.01, gamma=0.98):
    # Forward pass: Get hash codes from model
    batch_size = len(batch_labels)
    hash_codes = model(batch_features.to(device))  # shape: (batch_size, output_dim)
    random_hash_codes = model(random_batch_features.to(device))
    
    # 1. Bit balance term
    balance_loss = torch.mean(torch.abs(torch.sum(hash_codes, dim=1)))
    
    # 2. No sitting on the fence term
    # print(torch.abs(hash_codes))
    # print("###############")
    # print(torch.abs(hash_codes) - 1)
    # print("######################")
    # print(torch.norm(torch.abs(hash_codes) - 1, p=1, dim=1))
    # fence_loss = torch.mean(torch.sum(torch.abs(torch.abs(hash_codes) - 1),dim=1))
    fence_loss= torch.mean(torch.norm(torch.abs(hash_codes) - 1, p=1, dim=1))  # Apply the L1 norm


     
    # Create lists to store negative pairs
    negative_pairs = []
    for i in range(batch_size):
        for j in range(batch_size):
            if batch_labels[i] != random_batch_labels[j]:
                similarity = torch.nn.functional.cosine_similarity(
                    hash_codes[i].unsqueeze(0), random_hash_codes[j].unsqueeze(0))
                negative_pairs.append((i, j, similarity.item()))

    negative_pairs = sorted(negative_pairs, key=lambda x: x[2])
    negative_pairs = negative_pairs[:16]  # take top 16 negative pairs
    
    
    weak_supervision_loss = 0
    # Calculate the loss for sampled negative pairs
    if negative_pairs:
        weak_supervision_loss = torch.sum(
            torch.stack([torch.abs(torch.dot(hash_codes[u], hash_codes[v])) for u, v, _ in negative_pairs])
        ) / len(negative_pairs)
    
    # Get loss coefficients
    if use_hyperparameter:
        gamma = 1 - alpha - beta
    else:
        alpha, beta, gamma = model.get_loss_coefficients()

    # Calculate total loss
    total_loss = alpha * balance_loss + beta * fence_loss + gamma * weak_supervision_loss
    
    return total_loss, balance_loss, fence_loss, weak_supervision_loss


### Training Loop

In [42]:

def train_model(model, dataloader, optimizer, use_hyperparameter,alpha, beta, gamma,num_epochs=1, save_interval=1):
    model.train()  # set model to training mode
    model.to(device)
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Create a progress bar for the current epoch
        progress_bar = tqdm(dataloader, desc=f"Epoch [{epoch + 1}/{num_epochs}]", leave=False)
        
        for idx, batch in enumerate(progress_bar):
            batch_features = batch['features'].to(device)  # features from your dataset
            batch_labels = batch['label'].to(device)  # labels for the images
            # Random batch
            random_batch = get_random_batch()
            random_batch_features, random_batch_labels = random_batch['features'],random_batch['label']
            
            
            optimizer.zero_grad()

            # Calculate your custom loss
            loss, balance_loss, fence_loss, weak_supervision_loss = custom_loss_function(model, batch_features, batch_labels, random_batch_features, random_batch_labels,use_hyperparameter, alpha, beta, gamma)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Clamp and normalize loss coefficients
            model.clamp_and_normalize_loss_coefficients(min_value=0.01)

            # Track running loss
            running_loss += loss.item()
            
            # Update progress bar with loss values
            progress_bar.set_postfix({
                'loss': loss.item(),
                'balance_loss': balance_loss.item(),
                'fence_loss': fence_loss.item(),
                'weak_loss': weak_supervision_loss.item()
            })
            
        for i,j in model.named_parameters():
            if i == 'loss_coefficients':
                print(j)
        
        # Average loss per epoch
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        # Save the model every `save_interval` epochs
        if (epoch + 1) % save_interval == 0:
            model_save_path = f"imagenet_neural_lsh_1_epoch_{epoch + 1}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved at {model_save_path}")




In [43]:
device = 'cpu'

#### Hyper Parameters

In [46]:
input_dim = 1024  # Example input dimension
hash_bits = 16   # Example number of hash bits
alpha=0.01
beta=0.01
gamma=1- alpha-beta
# Instantiate the model
model = HashNetwork(input_dim=input_dim, hash_bits=hash_bits).to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-4)

#### Load model weights

In [47]:
# # Specify the path to the saved model
# model = HashNetwork(input_dim=input_dim, hash_bits=hash_bits).to(device)

# model_load_path = "imagenet_neural_lsh_1_epoch_10.pth"

# state_dict = torch.load(model_load_path, map_location=torch.device('cpu'))

# # Load the state dictionary into the model
# model.load_state_dict(state_dict)


#### Start Training model

In [48]:
train_model(model, train_dataloader, optimizer, num_epochs=10,use_hyperparameter=True, alpha=alpha, beta=beta, gamma=gamma)

Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [1/10], Loss: 0.2667


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [2/10], Loss: 0.1961


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [3/10], Loss: 0.1792


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [4/10], Loss: 0.1708


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [5/10], Loss: 0.1659


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [6/10], Loss: 0.1629


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [7/10], Loss: 0.1610


Parameter containing:
tensor([0.1206, 0.4311, 0.4483], requires_grad=True)
Epoch [8/10], Loss: 0.1599


KeyboardInterrupt: 

In [49]:
model_save_path = f"imagenet_neural_lsh_1_epoch_{8}.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved at {model_save_path}")

Model saved at imagenet_neural_lsh_1_epoch_8.pth


### Working on HASH

#### Define L and J

In [34]:
L = 3  # number of hash tables
J = 12 # number of bits to randomly sample from hash length

#### Hash Code on training data after training the model

In [35]:
hash_dict_list = []
position_list = []

# Iterate through the dataset
for i in range(L):
    hash_dict = dict()
     # Randomly select J unique positions from the range of the hash_length
    random_positions = random.sample(range(hash_bits), J)
    position_list.append(random_positions)
    
    for batch_idx, data in enumerate(tqdm(train_dataloader, desc="Processing Batches")):
        
        # Extract features and compute hash code
        feature = data['features'].to(device)
        hash_value = model(feature)
        hash_code = torch.where(hash_value > 0, 1, 0)
        
        # Convert the hash code tensor to a list of tuples for dictionary key compatibility
        hash_code_list = hash_code.tolist()

       
        # Iterate through each image and label in the batch
        for j in range(len(hash_code_list)):
            # Create a unique key from the hash code of each image
            hash_code_key = [hash_code_list[j][pos] for pos in random_positions]
            hash_code_key = tuple(hash_code_key)
            # Initialize the list if this hash code key is not yet in the dictionary
            if hash_code_key not in hash_dict:
                hash_dict[hash_code_key] = []
            
            # Append the tuple (batch index, image index, label) to the list of this hash code key
            val = [data['index'][j], data['label'][j].item()]
            hash_dict[hash_code_key].append(val)
    print(len(hash_dict.keys()))
    hash_dict_list.append(hash_dict)
    

# You now have a list of new hash tables with sampled bits, and a list of their corresponding positions


Processing Batches: 100%|██████████| 20019/20019 [10:22<00:00, 32.14it/s]


4096


Processing Batches: 100%|██████████| 20019/20019 [08:44<00:00, 38.15it/s]


4096


Processing Batches:  33%|███▎      | 6532/20019 [02:56<06:04, 36.95it/s]


KeyboardInterrupt: 

In [218]:
for batch_idx, data in enumerate(tqdm(train_dataloader, desc="Processing Batches")):
    
    # Extract features and compute hash code
    feature = data['features'].to(device)
    hash_value = model(feature)
    hash_code = torch.where(hash_value > 0, 1, 0)
    print(hash_code)
    
    # Convert the hash code tensor to a list of tuples for dictionary key compatibility
    hash_code_list = hash_code.tolist()

    break

Processing Batches:   0%|          | 0/782 [00:00<?, ?it/s]

tensor([[1, 1, 0,  ..., 1, 0, 0],
        [0, 0, 1,  ..., 1, 0, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]])


In [219]:
position_list

[[12, 4, 5, 1, 10, 7, 14, 8, 15, 6],
 [4, 6, 15, 5, 2, 3, 7, 14, 10, 0],
 [1, 13, 14, 6, 10, 8, 11, 0, 7, 3],
 [14, 2, 10, 8, 11, 1, 6, 13, 4, 0],
 [0, 8, 13, 12, 15, 14, 2, 10, 3, 5],
 [5, 14, 10, 0, 2, 15, 9, 11, 3, 13],
 [5, 9, 11, 3, 10, 7, 0, 14, 8, 1],
 [2, 8, 1, 0, 12, 5, 14, 9, 3, 13],
 [13, 1, 3, 5, 2, 14, 6, 8, 15, 0],
 [15, 5, 2, 0, 11, 7, 10, 6, 1, 14]]

#### Number of buckets in each hash table

In [220]:
for c,(i,j) in enumerate(hash_dict_list[1].items()):
    if c==15:
        print(i,j)
        break

(0, 0, 0, 0, 1, 1, 1, 1, 1, 1) [[48769, 8], [29304, 7], [17468, 4], [32509, 6], [24445, 1], [12664, 3], [7649, 4], [42964, 0], [5823, 2], [21998, 3], [46932, 6], [1079, 1], [38356, 5], [29723, 5], [4200, 2], [36139, 8], [5569, 2], [6716, 6], [49161, 6], [35127, 7], [34711, 8], [29135, 6], [11085, 3], [19846, 6], [22357, 5], [29258, 2], [12121, 7], [38276, 9], [19888, 0], [24138, 0], [28055, 8], [10643, 1], [37177, 9], [38597, 8], [7587, 8], [2235, 1], [8050, 6], [11780, 5], [43591, 3], [40994, 7], [6715, 1], [47489, 3], [43220, 3], [27810, 6], [13489, 1], [6043, 3]]


### Hash codes for test data

In [221]:
# Initialize a list to store similar images for each hyperplane
similar_images_list = []

# Iterate through each hyperplane
for i in range(L):
    # Create a dictionary to store similar images for this hyperplane
    similar_images = {}
    random_positions = position_list[i]
    # Process test data
    for batch_idx, data in enumerate(tqdm(test_dataloader, desc="Processing Test Batches")):
        feature = data['features'].to(device)
        

        # Compute hash values for the test data
        hash_value = model(feature)
        test_hash_code = torch.where(hash_value > 0, 1, 0)

        # Convert the hash code tensor to a list of tuples for dictionary key compatibility
        test_hash_code_list = test_hash_code.tolist()
        
        # Iterate through each test image
        for j in range(len(test_hash_code_list)):
            # Create a unique key from (batch_idx, i, label)
            test_key = (data['index'][j], data['label'][j].item())  # Convert tensor to tuple
            test_hash_code_key = [test_hash_code_list[j][pos] for pos in random_positions]
            # Create a unique key from the test hash code
            test_hash_code_key = tuple(test_hash_code_key)

            # Check if this hash code exists in the training hash dictionary
            if test_hash_code_key in hash_dict_list[i]:
                # Retrieve the corresponding images and their labels from the training hash dictionary
                similar_images[test_key] = hash_dict_list[i][test_hash_code_key]
            else:
                similar_images[test_key] = []

    # Store the similar images found for this hyperplane
    similar_images_list.append(similar_images)

# Now similar_images_list contains keys as (batch_idx, i, label) and values as similar images


Processing Test Batches: 100%|██████████| 157/157 [00:05<00:00, 26.63it/s]


In [222]:
for i,j in similar_images_list[0].items():
    print(j)
    break

[[23439, 3], [49261, 9], [25447, 1], [36372, 4], [38984, 8], [19267, 4], [38980, 5], [45887, 7], [32849, 4], [21109, 1], [45309, 2], [27850, 2], [20338, 0], [39326, 1], [38297, 8], [38097, 3], [432, 3], [33171, 9], [29840, 9], [46796, 8], [12366, 9], [43728, 3], [14811, 8], [36676, 5], [4643, 6], [35869, 4], [12376, 2], [33560, 9], [37218, 5], [41100, 3], [37293, 4], [19549, 5], [47154, 4], [45154, 0], [27019, 6], [36388, 6], [12328, 6], [45192, 3], [42280, 1], [20406, 2], [29966, 6], [17056, 8], [6793, 4], [21074, 4], [40494, 3], [11634, 0], [45410, 2], [40090, 2], [27372, 6], [16690, 3], [19937, 7], [34304, 8], [21730, 2], [10326, 3], [19427, 6], [45589, 0], [2605, 9], [23047, 2], [3170, 2], [23209, 6], [18897, 0], [15445, 5], [21086, 8], [7300, 3], [9013, 3], [1202, 8], [25257, 5], [12729, 2], [40291, 8], [28109, 3], [28356, 3], [13798, 9], [30454, 7], [44749, 9], [541, 3], [33102, 1], [24056, 0], [41278, 0], [27056, 0], [38274, 5], [40868, 4], [10194, 3], [24690, 4], [25352, 0], [2

### Create union of all hash list

In [223]:
# Initialize the dictionary to store similar images
similar_images_dict = {}

# Iterate through the similar_images_dict to populate similar_images_list
for i in similar_images_list:
    for j in tqdm(i.keys()):
        # Check if the key already exists in similar_images_list
        if j not in similar_images_list:
            similar_images_dict[j] = []  # Initialize the list if it doesn't exist

        similar_images_dict[j].extend(i[j])  # Deep copy of the elements

100%|██████████| 10000/10000 [00:00<00:00, 728721.79it/s]


In [224]:
# Initialize the dictionary to store similar images
unique_images_dict = {}

# Iterate through the similar_images_dict to populate unique_images_dict
for i, j in tqdm(similar_images_dict.items()):
    if i not in unique_images_dict:
        unique_images_dict[i] = []  # Initialize the list only if the key doesn't exist

    for k in j:
        if k not in unique_images_dict[i]:  # Check if 'k' is not already in the list
            unique_images_dict[i].append(k)  # Append 'k' to the list


100%|██████████| 10000/10000 [00:08<00:00, 1223.58it/s]


In [225]:
for k,(i,j) in enumerate(unique_images_dict.items()):
    print(j)
    if k==5:
        break

[[32417, 3], [18920, 7], [9709, 0], [14435, 8], [8503, 5], [6332, 5], [4315, 8], [8067, 7], [42321, 7], [29914, 2], [20558, 3], [37323, 7], [29332, 0], [48036, 7], [16642, 3], [42798, 8], [31412, 3], [34501, 8], [24503, 7], [44233, 8], [22671, 3], [21167, 7], [19926, 9], [49488, 8], [15042, 8], [26034, 2], [5374, 8], [1770, 2], [28916, 7], [6088, 9], [5858, 0], [18599, 8], [25369, 2], [6797, 9], [8003, 0], [3445, 2], [38844, 0], [24341, 0], [27588, 8], [32477, 8], [29785, 2], [17590, 5], [46757, 2], [25127, 3], [30645, 8], [16811, 7], [16603, 8], [6300, 2], [9092, 5], [41386, 7], [7287, 6], [15836, 7], [24162, 6], [14632, 8], [33992, 0], [45573, 8], [40796, 2], [26118, 8], [20817, 8], [24365, 0], [39837, 0], [9025, 9], [20787, 0], [40646, 2], [40303, 2], [13997, 0], [15540, 8], [28209, 9], [16243, 6], [19524, 8], [42036, 8], [16908, 9], [6438, 8], [19328, 8], [41376, 0], [25278, 3], [33795, 2], [15267, 0], [100, 6], [24846, 8], [28120, 9], [18461, 8], [44552, 0], [24132, 8], [18591, 9]

In [226]:
# Process each key in similar_images_dict with tqdm
for key, similar_images in tqdm(unique_images_dict.items(), desc="Processing keys"):
    
    test_feature = torch.tensor(ds_feature['test'][key[0]]['features'])
    # Process each similar image for the current key with tqdm
    for i, image in tqdm(enumerate(similar_images), desc=f"Processing similar images for key {key[0]}", leave=False):
        # Assuming image is a list/tuple with at least four elements
        train_feature = torch.tensor(ds_feature['train'][image[0]]['features'])
        
        # Calculate cosine similarity and store it
        similarity_score = F.cosine_similarity(test_feature, train_feature, dim=0)
        image = list(image[:2])  # Make a copy to modify
        # Append the similarity score to the copied list
        image.append(similarity_score.item())
        
        # Update the original list in similar_images
        similar_images[i] = image[:3]
        

Processing keys: 100%|██████████| 10000/10000 [12:16<00:00, 13.59it/s]


In [227]:
len(unique_images_dict.keys())

10000

### Evaluate

In [228]:
# Define functions to calculate Average Precision (AP) and Precision@K
def average_precision(predictions, targets):
    relevant_indices = (targets == 1).nonzero(as_tuple=True)[0]
    if len(relevant_indices) == 0:
        return 0.0

    precisions = []
    for i, idx in enumerate(relevant_indices, start=1):
        precision_at_i = (targets[:idx + 1].sum() / (idx + 1)).item()
        precisions.append(precision_at_i)

    return sum(precisions) / len(precisions)

def precision_at_k(predictions, targets, k):
    top_k_indices = torch.argsort(predictions, descending=True)[:k]
    top_k_relevant = targets[top_k_indices].sum().item()
    return top_k_relevant / k

# Prepare lists to store metric scores
map_scores = []
precision_10_scores = []
precision_50_scores = []

# Example: Iterate over your dataset to calculate metrics
for key, value in tqdm(unique_images_dict.items(), desc="Calculating metrics", total=len(similar_images_list)):
    if len(value) > 0:
        # Sort the values by similarity score (assuming v[4] is the similarity score)
        sorted_value = sorted(value, key=lambda x: x[-1], reverse=True)[:50]
        predictions = torch.tensor([v[2] for v in sorted_value])  # Similarity scores
        targets = torch.tensor([v[1] == key[1] for v in sorted_value], dtype=torch.float32)  # Relevance labels
        
        # Calculate Mean Average Precision for the current sample
        ap = average_precision(predictions, targets)
        map_scores.append(ap)
        
        # Calculate Precision@10 and Precision@50
        p10 = precision_at_k(predictions, targets, k=10)
        p50 = precision_at_k(predictions, targets, k=50)
        precision_10_scores.append(p10)
        precision_50_scores.append(p50)

# Calculate final average metrics
mean_avg_precision = sum(map_scores) / len(map_scores) if map_scores else 0.0
precision_10 = sum(precision_10_scores) / len(precision_10_scores) if precision_10_scores else 0.0
precision_50 = sum(precision_50_scores) / len(precision_50_scores) if precision_50_scores else 0.0

# Print results
print("Mean Average Precision (mAP):", mean_avg_precision)
print("Precision@10:", precision_10)
print("Precision@50:", precision_50)


Calculating metrics: 10000it [00:01, 6623.11it/s]          

Mean Average Precision (mAP): 0.7755063350586736
Precision@10: 0.6174917491749176
Precision@50: 0.3157375737573757
